# **BERT PAIR Relation Extraction Notebook**


This notebook contains the Bert-Pair approach to relation extraction. It should only be run after pretraining the model using the MTB_pretraining.ipynb file as we are trying to produce a hybrid model that is pretrained on MTB and then trained and evaluated on BERT-Pair.

## Importing files and libraries

In [1]:
!pip install transformers==3.0.0
!pip install ipython-autotime

%load_ext autotime

  Using cached transformers-3.0.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.8.0rc4.tar.gz (96 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached sentencepiece-0.2.0-cp39-cp39-win_amd64.whl.metadata (8.3 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
Using cached sentencepiece-0.2.0-cp39-cp39-win_amd64.whl (991 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      C:\Users\elsab\AppData\Local\Temp\pip-build-env-ikq2vhsv\overlay\Lib\site-packages\setuptools\dist.py:314: InformationOnly: Normalizing '0.8.0.rc4' to '0.8.0rc4'
        self.metadata.version = self._normalize_version(self.metadata.version)
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\tokenizers
      copying tokenizers\__init__.py -> build\lib.win-amd64-cpython-39\tokenizers
      creating build\lib.win-amd64-cpython-39\tokenizers\models
      copying tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-39\tokenizers\models
      creating build\lib.win-amd64-cpython-39\tokenizers\decoders
      copying tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-39\tokenizers

time: 0 ns (started: 2024-03-08 02:39:18 +00:00)


In [2]:
import os
import sys
import random
import json
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification


basepath = Path(os.getcwd())
sys.path.append(os.path.join(basepath, 'models'))
from framework import FewShotREFramework, FewShotREModel
import warnings
warnings.filterwarnings('ignore')

c:\Users\elsab\miniconda3\envs\data2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


time: 1.19 s (started: 2024-03-08 02:39:18 +00:00)


In [3]:
# defining paths for datasets and pretrained checkpoint file
checkpoint_path = os.path.join(basepath, "checkpoint_files","pretrain_checkpoint_BERT_1.pth.tar")

data_dir = os.path.join( os.getcwd(),'fewrel-training-data/')
train_file = os.path.join(data_dir, 'train_wiki')
val_file = os.path.join(data_dir, 'val_wiki')
test_file = os.path.join(data_dir, 'val_pubmed')


time: 0 ns (started: 2024-03-08 02:39:20 +00:00)


# BERT PAIR

This portion of the code is adapted from the Few_rel official repository, this contains the encoder, the model, and the data loader of the datasets. 

In [4]:
from torch import optim, nn
from torch.nn import functional as F

class BERTPAIRSentenceEncoder(nn.Module):
    def __init__(self, pretrain_path, max_length): 
        nn.Module.__init__(self)
        # we initialize the Bert model from huggingface with the pretrained bert-base-uncased 
        # The encoder is our main model
        self.bert = BertForSequenceClassification.from_pretrained(
                pretrain_path)
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def forward(self, inputs):
        # Encoder's main function, it passes the tokenified data to the bert model
        x = self.bert(inputs['word'], token_type_ids=inputs['seg'], attention_mask=inputs['mask'])[0]
        return x
    
    def tokenize(self, raw_tokens, pos_head, pos_tail):
        tokens = []
        cur_pos = 0
        pos1_in_index = 0
        pos2_in_index = 0
        for token in raw_tokens:
            token = token.lower()
            if cur_pos == pos_head[0]:
                tokens.append('[unused0]')
                pos1_in_index = len(tokens)
            if cur_pos == pos_tail[0]:
                tokens.append('[unused1]')
                pos2_in_index = len(tokens)
            tokens += self.tokenizer.tokenize(token)
            if cur_pos == pos_head[-1]:
                tokens.append('[unused2]')
            if cur_pos == pos_tail[-1]:
                tokens.append('[unused3]')
            cur_pos += 1
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokens)
        
        return indexed_tokens


class Pair(FewShotREModel):
    
    def __init__(self, sentence_encoder, hidden_size=8):
        FewShotREModel.__init__(self, sentence_encoder)
        self.hidden_size = hidden_size
        self.drop = nn.Dropout()

    def forward(self, batch, N, K, total_Q):
        '''
        support: Inputs of the support set.
        query: Inputs of the query set.
        N: Num of classes
        K: Num of instances for each class in the support set
        Q: Num of instances in the query set
        '''
    
        logits = self.sentence_encoder(batch)
        logits = logits.view(-1, total_Q, N, K, 2)
        logits = logits.mean(3) # (-1, total_Q, N, 2)

        logits_na, _ = logits[:, :, :, 0].min(2, keepdim=True) # (-1, totalQ, 1)
        logits = logits[:, : , :, 1] # (-1, total_Q, N)
        logits = torch.cat([logits, logits_na], 2) # (B, total_Q, N + 1)
        _, pred = torch.max(logits.view(-1, N+1), 1)
        return logits, pred


time: 0 ns (started: 2024-03-08 02:39:20 +00:00)


In [5]:
import torch.utils.data as data
# Official Data loader of the fewrel dataset, the datasets can be found in the folder fewrel-training-data
# under the form of json files.

class FewRelDatasetPair(data.Dataset):
    """
    FewRel Pair Dataset
    """
    def __init__(self, name, encoder, N, K, Q, na_rate, root, encoder_name):

        self.root = root
        path = os.path.join(root, name + ".json")
        if not os.path.exists(path):
            print("[ERROR] Data file does not exist!")
            assert(0)

        self.json_data = json.load(open(path))

        self.classes = list(self.json_data.keys())
        self.N = N
        self.K = K
        self.Q = Q
        self.na_rate = na_rate
        self.encoder = encoder
        self.encoder_name = encoder_name
        self.max_length = encoder.max_length

    def __getraw__(self, item):
        word = self.encoder.tokenize(item['tokens'],
            item['h'][2][0],
            item['t'][2][0])
        return word 

    def __additem__(self, d, word, pos1, pos2, mask):
        d['word'].append(word)
        d['pos1'].append(pos1)
        d['pos2'].append(pos2)
        d['mask'].append(mask)

    def __getitem__(self, index):
        target_classes = random.sample(self.classes, self.N)
        support = []
        query = []
        fusion_set = {'word': [], 'mask': [], 'seg': []}
        query_label = []
        Q_na = int(self.na_rate * self.Q)
        na_classes = list(filter(lambda x: x not in target_classes,  
            self.classes))

        for i, class_name in enumerate(target_classes):
            indices = np.random.choice(
                    list(range(len(self.json_data[class_name]))), 
                    self.K + self.Q, False)
            count = 0
            for j in indices:
                word  = self.__getraw__(
                        self.json_data[class_name][j])
                if count < self.K:
                    support.append(word)
                else:
                    query.append(word)
                count += 1

            query_label += [i] * self.Q

        # NA
        for j in range(Q_na):
            cur_class = np.random.choice(na_classes, 1, False)[0]
            index = np.random.choice(
                    list(range(len(self.json_data[cur_class]))),
                    1, False)[0]
            word = self.__getraw__(
                    self.json_data[cur_class][index])
            query.append(word)
        query_label += [self.N] * Q_na

        for word_query in query:
            for word_support in support:
                if self.encoder_name == 'bert':
                    SEP = self.encoder.tokenizer.convert_tokens_to_ids(['[SEP]'])
                    CLS = self.encoder.tokenizer.convert_tokens_to_ids(['[CLS]'])
                    word_tensor = torch.zeros((self.max_length)).long()
                else:
                    SEP = self.encoder.tokenizer.convert_tokens_to_ids(['</s>'])     
                    CLS = self.encoder.tokenizer.convert_tokens_to_ids(['<s>'])
                    word_tensor = torch.ones((self.max_length)).long()
                new_word = CLS + word_support + SEP + word_query + SEP
                for i in range(min(self.max_length, len(new_word))):
                    word_tensor[i] = new_word[i]
                mask_tensor = torch.zeros((self.max_length)).long()
                mask_tensor[:min(self.max_length, len(new_word))] = 1
                seg_tensor = torch.ones((self.max_length)).long()
                seg_tensor[:min(self.max_length, len(word_support) + 1)] = 0
                fusion_set['word'].append(word_tensor)
                fusion_set['mask'].append(mask_tensor)
                fusion_set['seg'].append(seg_tensor)

        return fusion_set, query_label
    
    def __len__(self):
        return 10000000
        
def collate_fn_pair(data):
    batch_set = {'word': [], 'seg': [], 'mask': []}
    batch_label = []
    fusion_sets, query_labels = zip(*data)
    for i in range(len(fusion_sets)):
        for k in fusion_sets[i]:
            batch_set[k] += fusion_sets[i][k]
        batch_label += query_labels[i]
    for k in batch_set:
        batch_set[k] = torch.stack(batch_set[k], 0)
    batch_label = torch.tensor(batch_label)
    return batch_set, batch_label

def get_loader_pair(name, encoder, N, K, Q, batch_size, 
        num_workers=0, collate_fn=collate_fn_pair, na_rate=0, root='./data', encoder_name='bert'):
    dataset = FewRelDatasetPair(name, encoder, N, K, Q, na_rate, root, encoder_name)
    data_loader = torch.utils.data.DataLoader(dataset=dataset,
            batch_size=batch_size,
            shuffle=False,
            pin_memory=True,
            num_workers=num_workers,
            collate_fn=collate_fn)
    return iter(data_loader)

time: 0 ns (started: 2024-03-08 02:39:20 +00:00)


## Training section

In [6]:
## Fine-Tuning and paths Parameters for the fewshot framework
trainN = 5
N = 5
K = 1
Q = 1
batch_size = 4
max_length = 3
hidden_size = 3
na_rate = 5

val_step = 200
train_iter = 500
val_iter = 100
test_iter = 1000

# Checkpoint files for the training phase
ckpt = os.path.join(basepath, 'checkpoint_files','bert-pair-fewrel.pth.tar')
prefix = 'bert-pair-fewrel.pth.tar'

time: 0 ns (started: 2024-03-08 02:39:20 +00:00)


In [7]:
# We initialize our encoder and model
sentence_encoder = BERTPAIRSentenceEncoder('bert-base-uncased', max_length)
model = Pair(sentence_encoder, hidden_size=hidden_size)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 1.31 s (started: 2024-03-08 02:39:20 +00:00)


Loading train-, validation- and test-data and initializing the FewShotREFramework with the different data loaders

In [8]:
# Loading our three datasets, one for training, one for validation and one for testing 
train_data_loader = get_loader_pair(train_file, sentence_encoder, N=trainN, K=K, Q=Q, na_rate=na_rate, batch_size=batch_size, encoder_name='bert')
val_data_loader = get_loader_pair(val_file, sentence_encoder, N=N, K=K, Q=Q, na_rate=na_rate, batch_size=batch_size, encoder_name='bert')
test_data_loader = get_loader_pair(test_file, sentence_encoder, N=N, K=K, Q=Q, na_rate=na_rate, batch_size=batch_size, encoder_name='bert')

# Initializing the few shot framework
framework = FewShotREFramework(train_data_loader, val_data_loader, test_data_loader)

time: 781 ms (started: 2024-03-08 02:39:21 +00:00)


Training the model using the provided FewShotREFramework from the authors of the FewRel dataset

In [9]:
# Training of the model using the few shot framework
# we load the mtb pretrained model on it first.

framework.train(model, prefix, batch_size, trainN, N, K, Q,
        pytorch_optim=optim.SGD, na_rate=na_rate, val_step=val_step, pair=True, 
        train_iter=train_iter, val_iter=val_iter, bert_optim=True,
        save_ckpt=ckpt, load_ckpt=checkpoint_path)

Start training...
Use bert optim!
Successfully loaded checkpoint 'c:\Users\elsab\Documents\git\text-mining\checkpoint_files\pretrain_checkpoint_BERT_1.pth.tar'
-------------------------------
Evaluation with evaluation data
[EVAL] step:  100 | accuracy: 12.90%
   Evaluation accuracy: 0.129000
Best checkpoint -> Saving checkpoint with accuracy of 0.129000
step:  100 | loss: 1.553072, accuracy: 46.67%
step:  200 | loss: 1.529634, accuracy: 48.33%
-------------------------------
Evaluation with evaluation data
[EVAL] step:  100 | accuracy: 50.00%
   Evaluation accuracy: 0.500000
Best checkpoint -> Saving checkpoint with accuracy of 0.500000
step:  300 | loss: 1.507452, accuracy: 50.00%
step:  400 | loss: 1.505783, accuracy: 50.00%
-------------------------------
Evaluation with evaluation data
[EVAL] step:  100 | accuracy: 50.00%
   Evaluation accuracy: 0.500000
step:  500 | loss: 1.500024, accuracy: 50.00%

####################

Finish training bert-pair-fewrel.pth.tar
Evaluation with Te